In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 30 16:22:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install wget
!pip install os
!pip install gzip
!pip install pickle
!pip install random
!pip install re
!pip install sys
!pip install torch
!pip install numpy

import torch
import numpy as np
import wget
import os
import gzip
import pickle
import random
import re
import sys

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=fc1339fc4c6d6ca256e6cf7daa5fbf8f6cde9b5ab67aa631760f444147d9c759
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)
ERROR: No matching distribution found for gzip
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [3]:
IMDB_URL = 'http://dlvu.github.io/data/imdb.{}.pkl.gz'
IMDB_FILE = 'imdb.{}.pkl.gz'

PAD, START, END, UNK = '.pad', '.start', '.end', '.unk'

def load_imdb(final=False, val=5000, seed=0, voc=None, char=False):

    cst = 'char' if char else 'word'

    imdb_url = IMDB_URL.format(cst)
    imdb_file = IMDB_FILE.format(cst)

    if not os.path.exists(imdb_file):
        wget.download(imdb_url)

    with gzip.open(imdb_file) as file:
        sequences, labels, i2w, w2i = pickle.load(file)

    if voc is not None and voc < len(i2w):
        nw_sequences = {}

        i2w = i2w[:voc]
        w2i = {w: i for i, w in enumerate(i2w)}

        mx, unk = voc, w2i['.unk']
        for key, seqs in sequences.items():
            nw_sequences[key] = []
            for seq in seqs:
                seq = [s if s < mx else unk for s in seq]
                nw_sequences[key].append(seq)

        sequences = nw_sequences

    if final:
        return (sequences['train'], labels['train']), (sequences['test'], labels['test']), (i2w, w2i), 2

    # Make a validation split
    random.seed(seed)

    x_train, y_train = [], []
    x_val, y_val = [], []

    val_ind = set( random.sample(range(len(sequences['train'])), k=val) )
    for i, (s, l) in enumerate(zip(sequences['train'], labels['train'])):
        if i in val_ind:
            x_val.append(s)
            y_val.append(l)
        else:
            x_train.append(s)
            y_train.append(l)

    return (x_train, y_train), \
           (x_val, y_val), \
           (i2w, w2i), 2


def gen_sentence(sent, g):

    symb = '_[a-z]*'

    while True:

        match = re.search(symb, sent)
        if match is None:
            return sent

        s = match.span()
        sent = sent[:s[0]] + random.choice(g[sent[s[0]:s[1]]]) + sent[s[1]:]

def gen_dyck(p):
    open = 1
    sent = '('
    while open > 0:
        if random.random() < p:
            sent += '('
            open += 1
        else:
            sent += ')'
            open -= 1

    return sent

def gen_ndfa(p):

    word = random.choice(['abc!', 'uvw!', 'klm!'])

    s = ''
    while True:
        if random.random() < p:
            return 's' + s + 's'
        else:
            s+= word

def load_brackets(n=50_000, seed=0):
    return load_toy(n, char=True, seed=seed, name='dyck')

def load_ndfa(n=50_000, seed=0):
    return load_toy(n, char=True, seed=seed, name='ndfa')

def load_toy(n=50_000, char=True, seed=0, name='lang'):

    random.seed(0)

    if name == 'lang':
        sent = '_s'

        toy = {
            '_s': ['_s _adv', '_np _vp', '_np _vp _prep _np', '_np _vp ( _prep _np )', '_np _vp _con _s' , '_np _vp ( _con _s )'],
            '_adv': ['briefly', 'quickly', 'impatiently'],
            '_np': ['a _noun', 'the _noun', 'a _adj _noun', 'the _adj _noun'],
            '_prep': ['on', 'with', 'to'],
            '_con' : ['while', 'but'],
            '_noun': ['mouse', 'bunny', 'cat', 'dog', 'man', 'woman', 'person'],
            '_vp': ['walked', 'walks', 'ran', 'runs', 'goes', 'went'],
            '_adj': ['short', 'quick', 'busy', 'nice', 'gorgeous']
        }

        sentences = [ gen_sentence(sent, toy) for _ in range(n)]
        sentences.sort(key=lambda s : len(s))

    elif name == 'dyck':

        sentences = [gen_dyck(7./16.) for _ in range(n)]
        sentences.sort(key=lambda s: len(s))

    elif name == 'ndfa':

        sentences = [gen_ndfa(1./4.) for _ in range(n)]
        sentences.sort(key=lambda s: len(s))

    else:
        raise Exception(name)

    tokens = set()
    for s in sentences:

        if char:
            for c in s:
                tokens.add(c)
        else:
            for w in s.split():
                tokens.add(w)

    i2t = [PAD, START, END, UNK] + list(tokens)
    t2i = {t:i for i, t in enumerate(i2t)}

    sequences = []
    for s in sentences:
        if char:
            tok = list(s)
        else:
            tok = s.split()
        sequences.append([t2i[t] for t in tok])

    return sequences, (i2t, t2i)

In [4]:
# load data
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False) # if final is True, train and test set is returned. Else validation data

print('x_train:', len(x_train))
print('y_train:', len(y_train))

x_train: 20000
y_train: 20000


In [5]:
def padding(x, y, w2i, batch_size = 16):

    batches_x = []
    batches_y = []

    # step over x met steps of batch_size
    for i in range(0, len(x), batch_size):

        start = i
        end = i + batch_size

        # get the batch
        batch_x = x[start:end]
        batch_y = y[start:end]

        batch = []
        for i, sentence in enumerate(batch_x):
            longest_sentence = max([len(sentence) for sentence in batch_x])
            if len(sentence) < longest_sentence:
                sentence += [w2i['pad']] * (longest_sentence - len(sentence))

            # print(len(sentence))
            batch.append(sentence)

        batches_x.append(batch)
        batches_y.append(batch_y)

    # transform all batches to tensors
    batches_x = [torch.tensor(batch, dtype = torch.long) for batch in batches_x]
    batches_y = [torch.tensor(batch, dtype = torch.long) for batch in batches_y]

    return batches_x, batches_y


In [6]:
# create batches
batch_size = 16
batches_x, batches_y = padding(x_train, y_train, w2i, batch_size)

In [7]:
class MLP(torch.nn.Module):

    def __init__(self, w2i, embedding_dim = 300, hidden_size = 300):
        super(MLP, self).__init__()
        num_embeddings = len(w2i)
        self.embedding =  torch.nn.Embedding(num_embeddings, embedding_dim)
        self.hidden = torch.nn.Linear(embedding_dim, hidden_size)
        self.output = torch.nn.Linear(hidden_size, 2)

    def forward(self, x):
        emb = self.embedding(x)
        k = self.hidden(emb)
        h = torch.nn.functional.relu(k)
        o, _ = torch.max(h, dim=1)
        y = self.output(o)
        return y


In [8]:
def train(batches_x, batches_y, model, epochs = 5, optimizer = 'Adam', lr=0.001):

    # Check if GPU is available and set the device accordingly
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device:", device)

    model = model.to(device)

    batches_x = [batch.to(device) for batch in batches_x]
    batches_y = [batch.to(device) for batch in batches_y]

    if optimizer == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(epochs):

        losses = []
        accuracies = []
        for i, batch in enumerate(batches_x):
            if i % 100 == 0: print('Batch number: ',i)
            predicted_y = model(batch)
            loss = torch.nn.functional.cross_entropy(predicted_y, batches_y[i])
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

            # get index of the max value (0 or 1)
            predicted_y = predicted_y.argmax(dim=1)

            # calculate accuracy: number of correct predictions / number of predictions
            n_correct = (predicted_y == batches_y[i]).sum().item()
            accuracy = n_correct / len(predicted_y)
            accuracies.append(accuracy)

        print('Epoch: ', epoch, 'Loss: ', np.mean(losses), 'Accuracy: ', np.mean(accuracies))

In [9]:
#model = MLP(w2i)
#train(batches_x, batches_y, model, epochs = 10, optimizer = 'Adam', lr=0.001)

In [13]:
class Elman(torch.nn.Module):
    def __init__(self, insize=300, outsize=300, hsize=300):
        super().__init__()
        self.lin1 = torch.nn.Linear(insize + hsize, hsize)  # Input-to-hidden layer
        self.lin2 = torch.nn.Linear(hsize, outsize)  # Hidden-to-output layer

    def forward(self, x, hidden=None):
        batch_size, sequence_size, embedding_size = x.size()
        if hidden is None:
            hidden = torch.zeros(batch_size, embedding_size, dtype=torch.float, device=x.device)  # Ensure hidden is on the same device

        outs = []
        for i in range(sequence_size):
            inp = torch.cat([x[:, i, :], hidden], dim=1)
            hidden = torch.nn.functional.relu(self.lin1(inp))
            out = self.lin2(hidden)
            outs.append(out[:, None, :])

        return torch.cat(outs, dim=1), hidden

In [14]:
class MLP2(torch.nn.Module):

    def __init__(self, w2i, embedding_dim = 300, hidden_size = 300):
        super(MLP2, self).__init__()
        num_embeddings = len(w2i)
        self.embedding =  torch.nn.Embedding(num_embeddings, embedding_dim)
        self.hidden = Elman(embedding_dim, hidden_size)
        self.output = torch.nn.Linear(hidden_size, 2)

    def forward(self, x):
        emb = self.embedding(x)
        tensors, hidden_layer = self.hidden(emb)
        h = torch.nn.functional.relu(tensors)
        o, _ = torch.max(h, dim=1)
        y = self.output(o)
        return y

In [ ]:
model = MLP2(w2i)
train(batches_x, batches_y, model, epochs = 10, optimizer = 'Adam', lr=0.001)

Using device: cuda
Batch number:  0
Batch number:  100
Batch number:  200
Batch number:  300
Batch number:  400
